In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'c:\\mlops project\\image-colorization-mlops'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelBuildingConfig:
    root_dir: Path
    KERNEL_SIZE_RES: int
    PADDING: int
    STRIDE: int
    BIAS: bool
    SCALE_FACTOR: int
    DIM: int
    DROPOUT_RATE: float
    KERNEL_SIZE_GENERATOR: int
    INPUT_CHANNELS: int
    OUTPUT_CHANNELS: int
    IN_CHANNELS: int



In [4]:
from src.imagecolorization.constants import *
from src.imagecolorization.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_building_config(self) -> ModelBuildingConfig:
        config = self.config.model_building
        params = self.params

        model_building_config = ModelBuildingConfig(
            root_dir=Path(config.root_dir),
            KERNEL_SIZE_RES=params.KERNEL_SIZE_RES,
            PADDING=params.PADDING,
            STRIDE=params.STRIDE,
            BIAS=params.BIAS,
            SCALE_FACTOR=params.SCALE_FACTOR,
            DIM=params.DIM,
            DROPOUT_RATE=params.DROPOUT_RATE,
            KERNEL_SIZE_GENERATOR=params.KERNEL_SIZE_GENERATOR,
            INPUT_CHANNELS=params.INPUT_CHANNELS,
            OUTPUT_CHANNELS=params.OUTPUT_CHANNELS,
            IN_CHANNELS=params.IN_CHANNELS
        )
        return model_building_config


In [5]:
import torch 
import torch.nn as nn
from pathlib import Path

class ResBlock(nn.Module):
    def __init__(self, in_channles, out_channels, stride = 1, kerenl_size = 3, padding = 1, bias = False):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(in_channles, out_channels, kernel_size=kerenl_size, padding=padding, stride=stride, bias = bias),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=kerenl_size, padding=padding, stride = 1, bias = bias),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
        self.identity_map = nn.Conv2d(in_channles, out_channels,kernel_size=1, stride=stride)
        self.relu = nn.ReLU(inplace= True)
        
    def forward(self, inputs):
        x = inputs.clone().detach()
        out = self.layer(x)
        residual = self.identity_map(inputs)
        skip  = out + residual
        return self.relu(skip)
    
    
class DownsampleConv(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1):
        super().__init__()
        self.layer = nn.Sequential(
            nn.MaxPool2d(2),
            ResBlock(in_channels, out_channels)
        )
        
    def forward(self, inputs):
        return self.layer(inputs)
    
    
    
class UpsampleConv(nn.Module):
    def __init__(self, in_channels, out_channels, scale_factor=2):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor,mode = 'bilinear', align_corners=True)
        self.res_block = ResBlock(in_channels + out_channels, out_channels)

    def forward(self, inputs, skip):
        x = self.upsample(inputs)
        x = torch.cat([x, skip], dim = 1)
        x = self.res_block(x)
        return x
    
class Generator(nn.Module):
    def __init__(self, input_channels, output_channels, dropout_rate = 0.2):
        super().__init__()
        self.encoding_layer1_= ResBlock(input_channels, 64)
        self.encoding_layer2_ = DownsampleConv(64, 128)
        self.encoding_layer3_ = DownsampleConv(128, 256)
        self.bridge = DownsampleConv(256, 512)
        self.decoding_layer3 = UpsampleConv(512, 256)
        self.decoding_layer2 = UpsampleConv(256, 128)
        self.decoding_layer1 = UpsampleConv(128 , 64)
        self.output = nn.Conv2d(64, output_channels, kernel_size = 1)
        self.dropout = nn.Dropout2d(dropout_rate)
        
    def forward(self, inputs):
        e1 = self.encoding_layer1_(inputs)
        e1 = self.dropout(e1)
        e2 = self.encoding_layer2_(e1)
        e2 = self.dropout(e2)
        e3 = self.encoding_layer3_(e2)
        e3 = self.dropout(e3)
        
        bridge = self.bridge(e3)
        bridge = self.dropout(bridge)
        
        d3 = self.decoding_layer3(bridge, e3)
        d2 =self.decoding_layer2(d3, e2)
        d1 = self.decoding_layer1(d2, e1)
        
        output = self.dropout(d1)
        return output
    
    
class Critic(nn.Module):
    def __init__(self, in_channels):
        super(Critic, self).__init__()

        def critic_block(in_filters, out_filters, normalization=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalization:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *critic_block(in_channels, 64, normalization=False),
            *critic_block(64, 128),
            *critic_block(128, 256),
            *critic_block(256, 512),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, 1)
        )

    def forward(self, img_input):
        output = self.model(img_input)
        return output
    

In [6]:
from torchsummary import summary
import torch
import os

class ModelBuilding:
    def __init__(self, config: ModelBuildingConfig):
        self.config = config
        self.root_dir = self.config.root_dir
        self.create_root_dir()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def create_root_dir(self):
        os.makedirs(self.root_dir, exist_ok=True)
        print(f"Created directory: {self.root_dir}")

    def get_generator(self):
        return Generator(
            input_channels=self.config.INPUT_CHANNELS,  # corrected argument name
            output_channels=self.config.OUTPUT_CHANNELS,  # corrected argument name
            dropout_rate=self.config.DROPOUT_RATE
        ).to(self.device)

    def get_critic(self):
        return Critic(in_channels=self.config.IN_CHANNELS).to(self.device)

    def build(self):
        generator = self.get_generator()
        critic = self.get_critic()
        return generator, critic

    def save_model(self, model, filename):
        path = self.root_dir / filename
        torch.save(model.state_dict(), path)
        print(f"Model saved to {path}")

    def display_summary(self, model, input_size):
        print(f"\nModel Summary:")
        summary(model, input_size)

    def build_and_save(self):
        generator, critic = self.build()

        # Display summaries
        print("\nGenerator Summary:")
        self.display_summary(generator, (self.config.INPUT_CHANNELS, 224, 224))  # Assuming input size is 224x224

        print("\nCritic Summary:")
        self.display_summary(critic, [(2, 224, 224), (1, 224, 224)])  # Critic takes two inputs: ab and l

        self.save_model(generator, "generator.pth")
        self.save_model(critic, "critic.pth")
        return generator, critic



In [7]:
try:
    config_manager = ConfigurationManager()
    model_config = config_manager.get_model_building_config()

    model_building = ModelBuilding(config=model_config)
    generator, critic = model_building.build_and_save()
except Exception as e:
    raise e

[2024-08-23 00:00:44,340: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-23 00:00:44,342: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-23 00:00:44,343: INFO: common: created directory at: artifacts]
Created directory: artifacts\model

Generator Summary:

Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             576
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,864
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
            Conv2d-7         [-1, 64, 224, 224]             128
              ReLU-8         [-1, 64, 224, 224]               0
          ResBlock-9         [-1, 64, 22